# Quickstart: Running DMFs as Quality Gate in ELT Pipeline

See the full blog-post from Jan Sommerfeld here on Medium: https://medium.com/snowflake/how-to-add-quality-checks-to-data-pipelines-using-the-new-snowflake-dmfs-e08b4174f3d9

Snowflake has released Data Metric Functions (DMFs) - a native solution to run a range of quality checks on your data (requires Enterprise edition or higher). Users can either choose from a growing library of system DMFs or write their own “UDMFs” with custom logic and thresholds.

Users use Tasks, a native orchestration capability, to schedule, modularize and orchestrate our ELT processing steps by connecting multiple Tasks to a Task Graph (aka DAG). Each Task runs a piece of code on a certain trigger and optionally a defined condition. Since Tasks can run almost anything (python, java, scala, sql, function, stored procedures, notebooks,…) they can also run Data Metric Functions. This allows us to integrate data quality checks deeply into our ingestion and transformation pipelines.

***With the following 6 steps we will set up a simple ELT data pipeline based on data quality checks that you can easily apply to your existing or next Task pipeline.***


## 1. Set up Demo Data Ingestion Stream

For simplicity we will just use the ACCOUNTADMIN role for this demo setup. If you don’t have it or want to use a separate role for this demo, you can check the Appendix at the end to grant all required privileges.
All following code will run in the context of this DEMO schema. So make sure you keep the context or use your own schema and warehouse.


In [ ]:
use role ACCOUNTADMIN;

create warehouse if not exists DEX_WH
    warehouse_size = XSMALL
    auto_suspend = 2;

create database if not exists DEX_DB;
create schema if not exists DEX_DB.DEMO;

Just to have a live demo we will first set up a Task that loads new rows into our source table to simulate a continuous ingestion. In your case that could be from a user interface, or something like sensor-data or analytics from a connector or some other database.

We will use some free weather data from the **Snowflake Marketplace**:
+ Go to Snowflake Marketplace 
+ Get the free **"Weather Source LLC: frostbyte"** data share
*(This data may be used in connection with the Snowflake Quickstart, but is provided solely by WeatherSource, and not by or on behalf of Snowflake.)*
+ Under "options rename the shared database "DEMO_WEATHER_DATA" just to shorten it

Now we can run the script below to create a Task that continuously loads small batches of data into a source table, while **intentionally adding some quality issues** to it.

In [ ]:
--- copy a sample of the data share into a new table 
create or replace table ALL_WEATHER_DATA
as
select
    ROW_NUMBER() over (order by DATE_VALID_STD desc, POSTAL_CODE) as ROW_ID,
    DATE_VALID_STD as DS,
    POSTAL_CODE as ZIPCODE,
    MIN_TEMPERATURE_AIR_2M_F as MIN_TEMP_IN_F,
    AVG_TEMPERATURE_AIR_2M_F as AVG_TEMP_IN_F,
    MAX_TEMPERATURE_AIR_2M_F as MAX_TEMP_IN_F,
from
    DEMO_WEATHER_DATA.ONPOINT_ID.HISTORY_DAY
where
    COUNTRY = 'US'
order by
    DATE_VALID_STD desc,
    POSTAL_CODE
limit 
    100000;

In [ ]:
--- continuously growing table with weather data as "external data source"
create or replace table CONTINUOUS_WEATHER_DATA(
    ROW_ID number,
    INSERTED timestamp,
    DS date,
    ZIPCODE varchar,
    MIN_TEMP_IN_F number,
    AVG_TEMP_IN_F number,
    MAX_TEMP_IN_F number
)
comment = 'Demo Source table'
;

In [ ]:
create or replace task ADD_WEATHER_DATA_TO_SOURCE
schedule = '5 minutes'
comment = 'adding 10 rows of weather data every 5 minutes and adding occasional anomalies'
as
begin
    if (
        (select 
            count(*)
        from 
            ALL_WEATHER_DATA A
        left join 
            CONTINUOUS_WEATHER_DATA C
            ON A.ROW_ID = C.ROW_ID
        where
            C.ROW_ID is NULL
        ) != 0 )
    then
        delete from CONTINUOUS_WEATHER_DATA;
    end if;
    
    insert into CONTINUOUS_WEATHER_DATA (
        ROW_ID,
        INSERTED,
        DS,
        ZIPCODE,
        MIN_TEMP_IN_F,
        AVG_TEMP_IN_F,
        MAX_TEMP_IN_F
    )
    select
        A.ROW_ID,
        current_timestamp() as INSERTED,
        A.DS,
        A.ZIPCODE as ZIPCODE,
--        case when A.ZIPCODE > 2000 then A.ZIPCODE else NULL end as ZIPCODE,
        A.MIN_TEMP_IN_F,
        A.AVG_TEMP_IN_F,
        case when uniform(1, 100, random()) != 1 then A.MAX_TEMP_IN_F else A.MAX_TEMP_IN_F * 8 end as MAX_TEMP_IN_F
    from 
        ALL_WEATHER_DATA A
    left join 
        CONTINUOUS_WEATHER_DATA C
        ON A.ROW_ID = C.ROW_ID
    where
        C.ROW_ID is NULL
    limit
        10;
        
end
;

In [ ]:
alter task ADD_WEATHER_DATA_TO_SOURCE resume;

## 2.  Setting up the demo transformation pipeline

For this demo setup we will use 4 tables:

* Source table - where new data comes in
* Landing table - where we load the new batch and run the quality checks on it
* Target table - for all “clean” data that meets expectations
* Quarantine table - for all “bad” data that failed expectations

The source table we already have from Step 2. So let’s create the other three:


In [ ]:
create or replace table RAW_WEATHER_DATA (
    ROW_ID number,
    INSERTED timestamp,
    DS date, 
    ZIPCODE varchar,
    MIN_TEMP_IN_F number,
    AVG_TEMP_IN_F number,
    MAX_TEMP_IN_F number
)
comment = 'Demo Landing table'
;

In [ ]:
create or replace table CLEAN_WEATHER_DATA (
    DS date, 
    ZIPCODE varchar,
    MIN_TEMP_IN_F number,
    AVG_TEMP_IN_F number,
    MAX_TEMP_IN_F number
)
comment = 'Demo Target table'
;

In [ ]:
create or replace table QUARANTINED_WEATHER_DATA (
    INSERTED timestamp,
    DS date, 
    ZIPCODE varchar,
    MIN_TEMP_IN_F number,
    AVG_TEMP_IN_F number,
    MAX_TEMP_IN_F number
)
comment = 'Demo Quarantine table'
;

Now we can build a **Task Graph** that runs whenever new data is added to the source table. 
So first we set up a Stream on the source table CONTINUOUS_WEATHER_DATA:

In [ ]:
create or replace stream NEW_WEATHER_DATA
    on table CONTINUOUS_WEATHER_DATA
    append_only = TRUE
    comment = 'checking for new weather data coming in'
;

Next we create the first Task to insert all new rows from the Stream into the landing table RAW_WEATHER_TABLE as soon as new data is available.

🔔 ***New Feature: “Triggered Tasks”** — We can simplify orchestration by omitting the schedule for our task and just set STREAM_HAS_DATA as a condition for the task to run.* 

In [ ]:
create or replace task LOAD_RAW_DATA
warehouse = 'DEX_WH'
when
    SYSTEM$STREAM_HAS_DATA('NEW_WEATHER_DATA')
as 
declare
    ROWS_LOADED number;
    RESULT_STRING varchar;
begin
    insert into RAW_WEATHER_DATA (
        ROW_ID,
        INSERTED,
        DS,
        ZIPCODE,
        MIN_TEMP_IN_F,
        AVG_TEMP_IN_F,
        MAX_TEMP_IN_F
    )
    select 
        ROW_ID,
        INSERTED,
        DS,
        ZIPCODE,
        MIN_TEMP_IN_F,
        AVG_TEMP_IN_F,
        MAX_TEMP_IN_F
    from 
        NEW_WEATHER_DATA
    ;

    --- to see number of rows loaded in the IU
    ROWS_LOADED := (select $1 from table(RESULT_SCAN(LAST_QUERY_ID())));
    RESULT_STRING := :ROWS_LOADED||' rows loaded into RAW_WEATHER_DATA';
    call SYSTEM$SET_RETURN_VALUE(:RESULT_STRING);
end;

**Task 2: Transformation**

This second task will run directly after the first task and simulate a transformation of the new dataset. In your case this might be much more complex. For our demo we keep it simple and just filter for the hot days with an average temperature over 68°F.

Once the new data is inserted into the target table CLEAN_WEATHER_DATA we empty the landing table again.


In [ ]:
create or replace task TRANSFORM_DATA
warehouse = 'DEX_WH'
after 
    LOAD_RAW_DATA
as 
begin
    insert into CLEAN_WEATHER_DATA (
            DS,
            ZIPCODE,
            MIN_TEMP_IN_F,
            AVG_TEMP_IN_F,
            MAX_TEMP_IN_F
            )
        select 
            DS,
            ZIPCODE,
            MIN_TEMP_IN_F,
            AVG_TEMP_IN_F,
            MAX_TEMP_IN_F
        from 
            RAW_WEATHER_DATA
        where
            AVG_TEMP_IN_F > 68
        ;
    delete from RAW_WEATHER_DATA;
end;

In [ ]:
-- lets just add one more to indicate the potential for further steps
create or replace task MORE_TRANSFORMATION
warehouse = 'DEX_WH'
after 
    TRANSFORM_DATA
as 
    select 
count(*) 
    from
        CLEAN_WEATHER_DATA
;

-- resume all Tasks of the graph
select SYSTEM$TASK_DEPENDENTS_ENABLE('LOAD_RAW_DATA');


Let’s switch to the Task Graph UI to 
* See the graph we created
* Check the run history to see if we have any errors
* check the return values for each Task

## 3. Assigning quality checks to the landing table

Let’s first have a look at all system Data Metric Functions that are already available by default. We can see them in Snowsight as Functions under the **SNOWFLAKE.CORE** schema or alternatively query for all DMFs in the account that our role is allowed to see:

In [ ]:
show data metric functions in account;

Now for our specific Demo dataset we want to also add a range-check to make sure that our temperature values are plausible and further data analysis from consumers downstream is not impacted by unrealistic values caused by faulty sensors.

For that we can write a UDMF (user-defined Data Metric Function) defining a range of plausible fahrenheit values:


In [ ]:
create or replace data metric function CHECK_FARENHEIT_PLAUSIBLE(
  TABLE_NAME table(
    COLUMN_VALUE number
  )
)
returns NUMBER
as
$$
  select
    count(*)
  from 
    TABLE_NAME
  where
    COLUMN_VALUE is not NULL
    and COLUMN_VALUE not between -40 and 140 
$$
;

We can now test our UDMF by test-running it manually on our source table:

In [ ]:
--- manually test-run the UDMF on our source table
select
    CHECK_FARENHEIT_PLAUSIBLE(          --- the UDMF
        select MAX_TEMP_IN_F            --- table column
        from CONTINUOUS_WEATHER_DATA    --- our source table
) as WRONG_FARENHEIT_VALUE
;

Now we can assign our UDMF together with a few system DMFs to our landing table:

In [ ]:
-- always set the schedule first
alter table RAW_WEATHER_DATA
    set DATA_METRIC_SCHEDULE = 'TRIGGER_ON_CHANGES';

    
--- assign DMFs to our RAW_WEATHER_DATA
alter table RAW_WEATHER_DATA
    add data metric function SNOWFLAKE.CORE.DUPLICATE_COUNT on (ROW_ID);

alter table RAW_WEATHER_DATA
    add data metric function SNOWFLAKE.CORE.NULL_COUNT on (DS);

alter table RAW_WEATHER_DATA
    add data metric function SNOWFLAKE.CORE.NULL_COUNT on (ZIPCODE);

-- add a custom DMF
alter table RAW_WEATHER_DATA
    add data metric function CHECK_FARENHEIT_PLAUSIBLE on (MAX_TEMP_IN_F);

The results of all scheduled checks performed by Data Metric Functions assigned to tables are stored in the view SNOWFLAKE.LOCAL.DATA_QUALITY_MONITORING_RESULTS. So we can query them or build us a simple Snowsight dashboard by running something like:

In [ ]:
select
    MEASUREMENT_TIME,
    METRIC_NAME,
    VALUE,
    TABLE_NAME,
    ARGUMENT_NAMES
from
    SNOWFLAKE.LOCAL.DATA_QUALITY_MONITORING_RESULTS
where
    TABLE_NAME = 'RAW_WEATHER_DATA'
    and TABLE_SCHEMA = 'DEMO'
order by
    MEASUREMENT_TIME desc
limit 
    1000;

## 4. Run DMFs as "Quality gate" part of the pipeline

Because we want our quality check Task to run all DMFs that are assigned to our landing table, even if we add or remove some DMFs later on, we don’t just want to call them explicitly from the Task. Instead we first build a helper function to modularize our code.

The function (UDTF) will accept a table name as argument and return all DMFs that are currently assigned to a column of this table.

In [ ]:
--- create a helper function to get all DMFs on a table

create or replace function GET_ACTIVE_QUALITY_CHECKS("TABLE_NAME" VARCHAR)
returns table(DMF VARCHAR, COL VARCHAR)
language SQL
as 
$$
    select 
        t1.METRIC_DATABASE_NAME||'.'||METRIC_SCHEMA_NAME||'.'||METRIC_NAME as DMF,
        REF.value:name ::string as COL
    from
        table(
            INFORMATION_SCHEMA.DATA_METRIC_FUNCTION_REFERENCES(
                REF_ENTITY_NAME => TABLE_NAME,
                REF_ENTITY_DOMAIN => 'table'
            )) as t1,
        table(flatten(input => parse_json(t1.REF_ARGUMENTS))) as REF    
    where
        SCHEDULE_STATUS = 'STARTED' 
$$
;

Before we call it within the Task, let’s test run it first:

In [ ]:
select DMF, COL from table(GET_ACTIVE_QUALITY_CHECKS('DEX_DB.DEMO.RAW_WEATHER_DATA'));

Now we can define a new Task to get all DMFs from this function and then run them all.

We store the result of each check in a TEST_RESULT variable and then sum them up in a RESULTS_SUMMARY variable.

This will give us the total of issues found from all checks and we can pass it on as output to the **Return value** of this Task. 

If our RESULT_SUMMARY remains ‘0’ then we know all checks have passed.


In [ ]:
-- suspend the graph so we can make changes
alter task LOAD_RAW_DATA suspend;

-- new task to run all DMFs on the landing table
create or replace task CHECK_DATA_QUALITY
warehouse = 'DEX_WH'
after 
    LOAD_RAW_DATA
as 
declare
    TEST_RESULT number;
    RESULTS_SUMMARY number default 0;
    RESULT_STRING varchar;
    c1 CURSOR for 
            --- get all DMFs and columns for active quality checks on this table by using the custom function 
                select DMF, COL from table(GET_ACTIVE_QUALITY_CHECKS('DEX_DB.DEMO.RAW_WEATHER_DATA'));
begin
    OPEN c1;
    --- looping throught all DMFs assigned to the table
    for REC in c1 DO

        --- manually run the DMF
        execute immediate 'select '||REC.DMF||'(select '||REC.COL||' from RAW_WEATHER_DATA);';  

        ---get the test result
        TEST_RESULT := (select $1 from table(RESULT_SCAN(LAST_QUERY_ID())));
                    
        -- Construct the results summary: if check did not pass then add issues to the counter
        if (:TEST_RESULT != 0)
            then RESULTS_SUMMARY := (:RESULTS_SUMMARY + :TEST_RESULT);
        end if;
    
    end for;
    CLOSE c1;

     --- construct result-string to act as condition for downstream tasks and to show number of quality issues found
    RESULT_STRING := (:RESULTS_SUMMARY||' separate quality issues found in table RAW_WEATHER_DATA');
    
    case when :RESULTS_SUMMARY = 0
    then
        call SYSTEM$SET_RETURN_VALUE('✅ All quality checks on RAW_WEATHER_DATA passed');
    else   
        call SYSTEM$SET_RETURN_VALUE(:RESULT_STRING);
    end;
end;

Now we just have to update our other transformation tasks to run AFTER the new quality check task.

And we are adding a condition to run ONLY if all quality checks have passed. For that we can use the Task return value as a condition.

🔔 ***New Feature: “Task Return Value as Condition”**  —  We can add a condition for a Child Task to run, based on the Return Value of a predecessor Task.*


In [ ]:
-- changing transformation task to now run after quality checks on only if all checks passed
alter task TRANSFORM_DATA remove after LOAD_RAW_DATA;

alter task TRANSFORM_DATA add after CHECK_DATA_QUALITY;

alter task TRANSFORM_DATA modify when SYSTEM$GET_PREDECESSOR_RETURN_VALUE('CHECK_DATA_QUALITY') = '✅ All quality checks on RAW_WEATHER_DATA passed';

-- resume all Tasks of the graph
select SYSTEM$TASK_DEPENDENTS_ENABLE('LOAD_RAW_DATA');

## 5. Isolate datasets with quality issues

Now we could just completely ignore the new dataset, clear the landing table and wait for the next one. More likely though we want to analyze that dataset and potentially even fix the data quality issues. To do that later we will first isolate this batch into our quarantine table.

So we add another Task to our graph and invert the condition so that it only runs when a quality check failed:

In [ ]:
-- suspend the graph so we can make changes
alter task LOAD_RAW_DATA suspend;

create or replace task ISOLATE_DATA_ISSUES
comment = 'isolate bad rows and clear landing table'
warehouse = 'DEX_WH'
after 
    CHECK_DATA_QUALITY
when 
    SYSTEM$GET_PREDECESSOR_RETURN_VALUE('CHECK_DATA_QUALITY') != '✅ All quality checks on RAW_WEATHER_DATA passed'
as 
begin
    insert into QUARANTINED_WEATHER_DATA (
            INSERTED,
            DS,
            ZIPCODE,
            MIN_TEMP_IN_F,
            AVG_TEMP_IN_F,
            MAX_TEMP_IN_F
            )
        select 
            INSERTED,
            DS,
            ZIPCODE,
            MIN_TEMP_IN_F,
            AVG_TEMP_IN_F,
            MAX_TEMP_IN_F
        from 
            RAW_WEATHER_DATA
        ;
    delete from RAW_WEATHER_DATA;
end;


-- resume all Tasks of the graph
select SYSTEM$TASK_DEPENDENTS_ENABLE('LOAD_RAW_DATA');

Now we can let this run, knowing that all batches with quality issues will be isolated and all batches that are good will be transformed further. Since we can not predict if and when this might happen, we want to finish this demo by adding a notification in case of quality issues.

## 6. Add notification about quality issues

Let us add another Task to our graph to send a notification when quality issues have been detected and rows were isolated. But maybe we know our data is not perfect and we don't want to get a notification every single time.

So let's use DMFs one more time to define a threshold and notify only when more than 1% of new weather data was quarantined. First we create a new UDMF to compare the number of rows in the quarantine table to those in the target table:

In [ ]:
--- create a custom DMF for comparing isolated rows vs clean rows
create or replace data metric function OVER_1PCT_ISOLATED_ROWS(
    TABLE_NAME table(
        DS date
    )
)
returns NUMBER
as
$$
  select
    case 
        when (select count(*) from QUARANTINED_WEATHER_DATA) > (select count(*) from CLEAN_WEATHER_DATA)
        then 1 
        else
            case when
                (select count(*) from QUARANTINED_WEATHER_DATA) * 100.0 / 
                (select count(*) from CLEAN_WEATHER_DATA) > 1
            then 1
            else 0
        end
    end
$$
;

Now we assign it to the quarantine table:

In [ ]:
-- always set the schedule first
alter table QUARANTINED_WEATHER_DATA
    set DATA_METRIC_SCHEDULE = 'TRIGGER_ON_CHANGES';

-- assign UDMF to QUARANTINED_WEATHER_DATA
alter table QUARANTINED_WEATHER_DATA
    add data metric function OVER_1PCT_ISOLATED_ROWS on (DS);

-- add a row-count system DMF for additional context 
alter table QUARANTINED_WEATHER_DATA
    add data metric function SNOWFLAKE.CORE.ROW_COUNT on ();

And now we can create another task that runs only if new rows were isolated and then checks if they surpass the 1% threshold and only then sends us a notification.

In [ ]:
alter task LOAD_RAW_DATA suspend;

create or replace task NOTIFY_ABOUT_QUALITY_ISSUE
warehouse = 'DEX_WH'
after 
    ISOLATE_DATA_ISSUES
as 
declare
    TEST_RESULT integer;
begin

    TEST_RESULT := (select OVER_1_PERCENT from(
        select OVER_1PCT_ISOLATED_ROWS( select DS from QUARANTINED_WEATHER_DATA)as OVER_1_PERCENT
        )
    );

    case when :TEST_RESULT > 0 then
        call SYSTEM$SEND_SNOWFLAKE_NOTIFICATION(
            SNOWFLAKE.NOTIFICATION.TEXT_HTML(
                'More than 1 percent of new weather data was quarantined due to data quality issues.'   -- my html message for emails
            ),       
            SNOWFLAKE.NOTIFICATION.EMAIL_INTEGRATION_CONFIG(
                'YOUR_EMAIL_NOTIFICATION_INTEGRATION',             -- email integration
                'Snowflake DEMO Pipeline Alert',                   -- email header
                ARRAY_CONSTRUCT('YOUR_EMAIL_HERE')                 -- validated user email addresses
            )       
        );

        call SYSTEM$SET_RETURN_VALUE('Over 1% bad rows. Notification sent to YOUR_EMAIL_NOTIFICATION_INTEGRATION');
            
    else   
        call SYSTEM$SET_RETURN_VALUE('Less than 1% bad rows. No notification sent.');
    end;
end;

In [ ]:
-- resume all Tasks of the graph
select SYSTEM$TASK_DEPENDENTS_ENABLE('LOAD_RAW_DATA');

With this dependency setup we are also reducing redundant notifications, as they will only trigger when new quality issues are detected and the percentage of bad rows is still above 1%.

Once our Task Graph had a few runs we can now also see the 2 different paths that can occur. 
Navigate to **Monitoring / Task History** and filter to our DEX_DB/DEMO schema and our LOAD_RAW_DATA root task to see the history of graph runs. 

We can see they are all successful, as they are handling both cases (quality checks passed or failed).

Selecting a run from the History list we will mostly see graphs where the checks passed and data was processed mixed with a few occasional runs that did detect quality issues and isolated the dataset instead.


## Now make it yours!

While this setup should be generic enough for you to apply to your existing ELT Task graphs there are many opportunities for you to further customize and automate this according to your needs.
+ You can start by writing and running your own DMFs. 
+ You can customize the notifications logic and message content.
+ Or you can Automatically process the isolated rows by adding more Tasks to the isolated data branch of the graph that can delete, sanitize or extrapolate data and then merge it back into the clean-data table.
+ Or we add a Streamlit App with a data-editor for a data expert to manually review and correct the isolated rows before merging them…


## Appendix

**Official Snowflake documentation:**

+ https://docs.snowflake.com/en/user-guide/data-quality-intro
+ https://docs.snowflake.com/en/user-guide/tasks-intro
+ https://docs.snowflake.com/en/user-guide/tasks-intro#label-tasks-triggered  
+ https://docs.snowflake.com/en/sql-reference/functions/system_set_return_value 
+ https://docs.snowflake.com/en/sql-reference/functions/system_get_predecessor_return_value 


**Granting required role privileges**

+ if you don't want to use the ACCOUNTADMIN role, then create a new role and grant all required privileges for this setup

In [ ]:
create role if not exists DEMO_USER;
grant role DEMO_USER to user YOUR_USERNAME;         -- insert your username here

grant create table on schema DEX_DB.DEMO to role DEMO_USER;
grant create stream on schema DEX_DB.DEMO to role DEMO_USER;
grant create task on schema DEX_DB.DEMO to role DEMO_USER;
grant create function on schema DEX_DB.DEMO to role DEMO_USER;

grant usage on warehouse DEX_WH to role DEMO_USER;

-- to create notification integrations (optional)
grant create integration on account to role DEMO_USER;
 
-- to create and run data metrics functions and see their results
grant create data metric function on schema DEX_DB.DEMO to role DEMO_USER;
grant execute data metric function on account to role DEMO_USER;
grant application role SNOWFLAKE.DATA_QUALITY_MONITORING_VIEWER to role DEMO_USER;

use role DEMO_USER;